![Two data scientists working on a dashboard.](hr-image-small.png)

A common problem when creating models to generate business value from data is that the datasets can be so large that it can take days for the model to generate predictions. Ensuring that your dataset is stored as efficiently as possible is crucial for allowing these models to run on a more reasonable timescale without having to reduce the size of the dataset.

You've been hired by a major online data science training provider called *Training Data Ltd.* to clean up one of their largest customer datasets. This dataset will eventually be used to predict whether their students are looking for a new job or not, information that they will then use to direct them to prospective recruiters.

You've been given access to `customer_train.csv`, which is a subset of their entire customer dataset, so you can create a proof-of-concept of a much more efficient storage solution. The dataset contains anonymized student information, and whether they were looking for a new job or not during training:

| Column                   | Description                                                                      |
|------------------------- |--------------------------------------------------------------------------------- |
| `student_id`             | A unique ID for each student.                                                    |
| `city`                   | A code for the city the student lives in.                                        |
| `city_development_index` | A scaled development index for the city.                                         |
| `gender`                 | The student's gender.                                                            |
| `relevant_experience`    | An indicator of the student's work relevant experience.                          |
| `enrolled_university`    | The type of university course enrolled in (if any).                              |
| `education_level`        | The student's education level.                                                   |
| `major_discipline`       | The educational discipline of the student.                                       |
| `experience`             | The student's total work experience (in years).                                  |
| `company_size`           | The number of employees at the student's current employer.                       |
| `company_type`           | The type of company employing the student.                                       |
| `last_new_job`           | The number of years between the student's current and previous jobs.             |
| `training_hours`         | The number of hours of training completed.                                       |
| `job_change`             | An indicator of whether the student is looking for a new job (`1`) or not (`0`). |

In [1]:
# Import necessary libraries
import pandas as pd

# Load the dataset
ds_jobs = pd.read_csv("customer_train.csv")

# View the dataset
ds_jobs.head()

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
0,8949,city_103,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevant experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevant experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
3,33241,city_115,0.789,NaN,No relevant experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,666,city_162,0.767,Male,Has relevant experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0


In [2]:
# Create a copy of ds_jobs for transforming
ds_jobs_transformed = ds_jobs.copy()

# Start coding here. Use as many cells as you like!

In [3]:
ds_jobs.memory_usage()

Index                        128
student_id                153264
city                      153264
city_development_index    153264
gender                    153264
relevant_experience       153264
enrolled_university       153264
education_level           153264
major_discipline          153264
experience                153264
company_size              153264
company_type              153264
last_new_job              153264
training_hours            153264
job_change                153264
dtype: int64

In [4]:
ds_jobs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   student_id              19158 non-null  int64  
 1   city                    19158 non-null  object 
 2   city_development_index  19158 non-null  float64
 3   gender                  14650 non-null  object 
 4   relevant_experience     19158 non-null  object 
 5   enrolled_university     18772 non-null  object 
 6   education_level         18698 non-null  object 
 7   major_discipline        16345 non-null  object 
 8   experience              19093 non-null  object 
 9   company_size            13220 non-null  object 
 10  company_type            13018 non-null  object 
 11  last_new_job            18735 non-null  object 
 12  training_hours          19158 non-null  int64  
 13  job_change              19158 non-null  float64
dtypes: float64(2), int64(2), object(10)
me

In [5]:
# type check

new_types = ['bool','int32','float16','category']

for field in ds_jobs_transformed:
    if ds_jobs_transformed[field].dtype not in new_types:
        print(ds_jobs_transformed[field].value_counts())
        print(field, 'Dtype =', ds_jobs[field].dtype)
        print('-----------------------')

8949     1
10660    1
30726    1
18507    1
31273    1
        ..
11547    1
32067    1
14356    1
18051    1
23834    1
Name: student_id, Length: 19158, dtype: int64
student_id Dtype = int64
-----------------------
city_103    4355
city_21     2702
city_16     1533
city_114    1336
city_160     845
            ... 
city_129       3
city_111       3
city_121       3
city_140       1
city_171       1
Name: city, Length: 123, dtype: int64
city Dtype = object
-----------------------
0.920    5200
0.624    2702
0.910    1533
0.926    1336
0.698     683
         ... 
0.649       4
0.807       4
0.781       3
0.625       3
0.664       1
Name: city_development_index, Length: 93, dtype: int64
city_development_index Dtype = float64
-----------------------
Male      13221
Female     1238
Other       191
Name: gender, dtype: int64
gender Dtype = object
-----------------------
Has relevant experience    13792
No relevant experience      5366
Name: relevant_experience, dtype: int64
relevant_experie

In [6]:
# decimal check

for field in ds_jobs:
    if ds_jobs[field].dtype not in ['object']:
        print(field,(ds_jobs[field] % 1 != 0).any())
        if ((ds_jobs[field] % 1 != 0).any()):
            print(ds_jobs[field])

student_id False
city_development_index True
0        0.920
1        0.776
2        0.624
3        0.789
4        0.767
         ...  
19153    0.878
19154    0.920
19155    0.920
19156    0.802
19157    0.855
Name: city_development_index, Length: 19158, dtype: float64
training_hours False
job_change False


In [7]:
# Create a mapping dictionary of columns containing two-factor categories to convert to Booleans

two_factor_cats = {
    'relevant_experience': {'No relevant experience': False, 'Has relevant experience': True},
    'job_change': {0.0: False, 1.0: True}
}

two_factors_columns = ['job_change', 'relevant_experience']

for col in two_factors_columns:
    ds_jobs_transformed[col] = ds_jobs_transformed[col].map(two_factor_cats[col])


In [8]:
# Columns containing integers only must be stored as 32-bit integers (int32).

bit_32_integers = ['student_id','training_hours']

ds_jobs_transformed[bit_32_integers] = ds_jobs_transformed[bit_32_integers].astype('int32')

In [9]:
# Columns containing floats must be stored as 16-bit floats (float16).

float_16_integers = ['city_development_index']

ds_jobs_transformed[float_16_integers] = ds_jobs_transformed[float_16_integers].astype('float16')

In [10]:
# Columns containing nominal categorical (not ordinal) data must be stored as the category data type.

cat_non_ordinal = ['company_type','major_discipline','gender','city']

ds_jobs_transformed[cat_non_ordinal] = ds_jobs_transformed[cat_non_ordinal].astype('category')

In [11]:
# Columns containing ordinal categorical data must be stored as ordered categories, and not mapped to numerical values, with an order that reflects the natural order of the column.

# enrolled_university
enrolled_university_order = ['no_enrollment', 'Part time course', 'Full time course']

ds_jobs_transformed['enrolled_university'] = pd.Categorical(ds_jobs_transformed['enrolled_university'], categories=enrolled_university_order, ordered=True)

# education_level
education_level_order = ['Primary School', 'High School', 'Graduate', 'Masters', 'Phd']

ds_jobs_transformed['education_level'] = pd.Categorical(ds_jobs_transformed['education_level'], categories=education_level_order, ordered=True)

# last_new_job
last_new_job_order = ['never', '1', '2', '3', '4', '>4']

ds_jobs_transformed['last_new_job'] = pd.Categorical(ds_jobs_transformed['last_new_job'], categories=last_new_job_order, ordered=True)

# company_size
company_size_order = ['<10', '10-49', '50-99', '100-499', '500-999', '1000-4999', '5000-9999', '10000']

ds_jobs_transformed['company_size'] = pd.Categorical(ds_jobs_transformed['company_size'], categories=company_size_order, ordered=True)


In [12]:
# field experience: creating bins and transform to categorical

# Convert the 'experience' column to numeric, handling special cases
ds_jobs_transformed['experience'] = ds_jobs_transformed['experience'].replace({'<1': 0, '>20': 99})

ds_jobs_transformed['experience'] = \
pd.to_numeric(ds_jobs_transformed['experience'], errors='coerce').fillna(99).astype(int)
          
# Define the bins
bins = [-1, 1, 4, 9, 14, 19, 999]
labels = ['<1', '1-4', '5-9', '10-14', '15-19', '>19']

# Create bins and convert to categorical
ds_jobs_transformed['experience'] = pd.cut(ds_jobs_transformed['experience'], bins=bins, labels=labels, right=False)


In [13]:
ds_jobs_transformed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   student_id              19158 non-null  int32   
 1   city                    19158 non-null  category
 2   city_development_index  19158 non-null  float16 
 3   gender                  14650 non-null  category
 4   relevant_experience     19158 non-null  bool    
 5   enrolled_university     18772 non-null  category
 6   education_level         18698 non-null  category
 7   major_discipline        16345 non-null  category
 8   experience              19158 non-null  category
 9   company_size            11201 non-null  category
 10  company_type            13018 non-null  category
 11  last_new_job            18735 non-null  category
 12  training_hours          19158 non-null  int32   
 13  job_change              19158 non-null  bool    
dtypes: bool(2), category(9

In [14]:
ds_jobs_transformed.memory_usage()

Index                       128
student_id                76632
city                      24310
city_development_index    38316
gender                    19290
relevant_experience       19158
enrolled_university       19290
education_level           19370
major_discipline          19378
experience                19378
company_size              19522
company_type              19378
last_new_job              19378
training_hours            76632
job_change                19158
dtype: int64

In [15]:
for field in ds_jobs_transformed:
    if ds_jobs_transformed[field].dtype == 'category':
        print(ds_jobs_transformed[field].value_counts().sort_index())
        print('----------------------------------------')

city_1       26
city_10      86
city_100    275
city_101     75
city_102    304
           ... 
city_93      26
city_94      26
city_97     104
city_98      79
city_99      94
Name: city, Length: 123, dtype: int64
----------------------------------------
Female     1238
Male      13221
Other       191
Name: gender, dtype: int64
----------------------------------------
no_enrollment       13817
Part time course     1198
Full time course     3757
Name: enrolled_university, dtype: int64
----------------------------------------
Primary School      308
High School        2017
Graduate          11598
Masters            4361
Phd                 414
Name: education_level, dtype: int64
----------------------------------------
Arts                 253
Business Degree      327
Humanities           669
No Major             223
Other                381
STEM               14492
Name: major_discipline, dtype: int64
----------------------------------------
<1        522
1-4      3030
5-9      5879
10-

In [16]:
ds_jobs_transformed.head()

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
0,8949,city_103,0.919922,Male,True,no_enrollment,Graduate,STEM,>19,NaN,NaN,1,36,True
1,29725,city_40,0.775879,Male,False,no_enrollment,Graduate,STEM,15-19,50-99,Pvt Ltd,>4,47,False
2,11561,city_21,0.624023,NaN,False,Full time course,Graduate,STEM,5-9,NaN,NaN,never,83,False
3,33241,city_115,0.789062,NaN,False,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,True
4,666,city_162,0.767090,Male,True,no_enrollment,Masters,STEM,>19,50-99,Funded Startup,4,8,False


In [17]:
ds_jobs_transformed = ds_jobs_transformed[(ds_jobs_transformed['experience']>'5-9') & (ds_jobs_transformed['company_size']>'500-999')]

In [18]:
ds_jobs_transformed['experience'].value_counts()

>19      507
10-14    388
15-19    312
<1         0
1-4        0
5-9        0
Name: experience, dtype: int64

In [19]:
ds_jobs_transformed['company_size'].value_counts()

1000-4999    865
5000-9999    342
<10            0
10-49          0
50-99          0
100-499        0
500-999        0
10000          0
Name: company_size, dtype: int64

In [20]:
# ###########################
# # Another Solution Below:
# ###########################


# # Import necessary libraries
# import pandas as pd

# # Load the dataset and create a copy
# ds_jobs = pd.read_csv("customer_train.csv")
# ds_jobs_transformed = ds_jobs.copy()

# # EDA to help identify ordinal, nominal, and two-factor categories
# for col in ds_jobs.select_dtypes("object").columns:
#     print(ds_jobs_transformed[col].value_counts(), '\n')

# # Create a dictionary of columns containing ordered categorical data
# ordered_cats = {
#     'enrolled_university': ['no_enrollment', 'Part time course', 'Full time course'],
#     'education_level': ['Primary School', 'High School', 'Graduate', 'Masters', 'Phd'],
#     'experience': ['<1'] + list(map(str, range(1, 21))) + ['>20'],
#     'company_size': ['<10', '10-49', '50-99', '100-499', '500-999', '1000-4999', '5000-9999', '10000+'],
#     'last_new_job': ['never', '1', '2', '3', '4', '>4']
# }

# # Create a mapping dictionary of columns containing two-factor categories to convert to Booleans
# two_factor_cats = {
#     'relevant_experience': {'No relevant experience': False, 'Has relevant experience': True},
#     'job_change': {0.0: False, 1.0: True}
# }

# # Loop through DataFrame columns to efficiently change data types
# for col in ds_jobs_transformed:
    
#     # Convert two-factor categories to bool
#     if col in ['relevant_experience', 'job_change']:
#         ds_jobs_transformed[col] = ds_jobs_transformed[col].map(two_factor_cats[col])
    
#     # Convert integer columns to int32
#     elif col in ['student_id', 'training_hours']:
#         ds_jobs_transformed[col] = ds_jobs_transformed[col].astype('int32')
    
#     # Convert float columns to float16
#     elif col == 'city_development_index':
#         ds_jobs_transformed[col] = ds_jobs_transformed[col].astype('float16')
    
#     # Convert columns containing ordered categorical data to ordered categories using dict
#     elif col in ordered_cats.keys():
#         category = pd.CategoricalDtype(ordered_cats[col], ordered=True)
#         ds_jobs_transformed[col] = ds_jobs_transformed[col].astype(category)
    
#     # Convert remaining columns to standard categories
#     else:
#         ds_jobs_transformed[col] = ds_jobs_transformed[col].astype('category')
        
# # Filter students with 10 or more years experience at companies with at least 1000 employees
# ds_jobs_transformed = ds_jobs_transformed[(ds_jobs_transformed['experience'] >= '10') & (ds_jobs_transformed['company_size'] >= '1000-4999')]